In [ ]:
%pip install -U sagemaker

In [ ]:
import sys  
sys.path.insert(1, './code/')
import config


In [ ]:
import datetime
import time
import tarfile

import boto3
import pandas as pd
import numpy as np
from sagemaker import get_execution_role
import sagemaker


sm_boto3 = boto3.client("sagemaker")

sess = sagemaker.Session()

region = sess.boto_session.region_name

# We use the Estimator from the SageMaker Python SDK
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"
env = {
    'SAGEMAKER_REQUIREMENTS': 'requirements.txt', # path relative to `source_dir` below.
}

sklearn_estimator = SKLearn(
    entry_point=config.script_name, #Chooses the appropriate training script. Change the model in the config.py file accordingly
    role=get_execution_role(),
    env=env,
    instance_count=1,
    source_dir = 'code',
#     instance_type="ml.g5.8xlarge",
    instance_type="ml.c5.18xlarge", #DEFINES WHAT EC2 INSTANCE THE TRAINING JOB IS RUN ON
    framework_version=FRAMEWORK_VERSION,
    py_version='py3',
    base_job_name="xgb-xgboost",#NAME OF BASE JOB 
    test_version = 'first-test'
)

In [ ]:
bucket = 'halton-ubc'
training_path = 'data/'

data_location = 's3://{}/{}'.format(bucket, training_path)

sklearn_estimator.fit(data_location)

In [ ]:
# After training, you can move just the model.tar.gz to your desired location
def move_model_artifact(estimator, desired_s3_path):
    import boto3
    s3 = boto3.client('s3')
    
    # Get the source bucket and key from the model data path
    source_uri = estimator.model_data
    source_bucket = source_uri.split('/')[2]
    source_key = '/'.join(source_uri.split('/')[3:])
    
    # Get the destination bucket and key
    dest_bucket = desired_s3_path.split('/')[2]
    dest_key = '/'.join(desired_s3_path.split('/')[3:]) + 'model.tar.gz'
    
    # Copy the model artifact
    s3.copy_object(
        CopySource={'Bucket': source_bucket, 'Key': source_key},
        Bucket=dest_bucket,
        Key=dest_key
    )
    
#     # Optionally, delete the original files if needed
#     response = s3.list_objects_v2(Bucket=source_bucket, Prefix='/'.join(source_key.split('/')[:-1]))
#     for obj in response.get('Contents', []):
#         s3.delete_object(Bucket=source_bucket, Key=obj['Key'])


# After training, move just the model artifact
move_model_artifact(sklearn_estimator, 's3://halton-ubc/model/')